In [2]:
import os, shutil, glob
from pathlib import Path
from os.path import join

import pandas as pd
import numpy as np

import anndata as ad
import scanpy as sc

#import pdb

In [3]:
#Paths
#=====

opath= r'C:\Users\moren\PycharmProjects\TFM\R\ELA_DFT\anndata_ELA_DFT'

In [4]:
#Load matrix count
#==================

ifile_count = r"C:\Users\moren\PycharmProjects\TFM\R\ELA_DFT\input\ALL_FTD_S2_FINAL_CUT_CUT_FINAL.csv"
df = pd.read_table(ifile_count, header=0)

#Traspose original data
m_count = pd.DataFrame.transpose(df)

#Adding headings
m_count.columns = m_count.iloc[0]
m_count = m_count[1:]

m_count.head(10)

Geneid,DDX11L1,WASH7P,MIR6859-1,MIR1302-2HG,MIR1302-2,FAM138A,OR4G4P,OR4G11P,OR4F5,AL627309.1,...,RNA5-8SN4,AL354822.1,AC023491.2,AC023491.1,AC004556.1,AC233755.2,AC233755.1,AC240274.1,AC213203.2,AC213203.1
long10BLACK,0,77,2,1,0,0,0,0,0,8,...,0,61,0,0,53,0,0,507,0,0
long11BLACK,0,38,7,1,0,0,0,0,0,7,...,0,56,0,0,0,0,0,556,0,0
long12BLACK,0,37,4,0,0,0,0,0,0,0,...,0,175,0,0,0,0,0,432,1,0
long13BLACK,1,34,5,0,0,0,0,0,0,1,...,0,252,0,0,0,0,0,328,0,0
long14BLACK,0,31,1,0,0,0,0,0,0,2,...,0,12,0,0,2,0,0,163,0,0
long15BLACK,0,27,1,0,0,0,0,0,0,11,...,0,39,1,0,0,0,0,549,0,0
long16BLACK,0,24,5,0,0,0,0,0,0,8,...,0,249,0,0,2,0,0,416,0,0
long17BLACK,0,59,0,0,0,0,0,0,0,6,...,0,262,0,0,1,0,0,868,0,0
long18BLACK,0,54,7,0,0,0,0,0,0,1,...,0,17,0,0,1,0,0,697,0,0
long19BLACK,0,54,8,1,0,0,0,0,0,4,...,0,19,0,0,0,0,0,353,0,0


In [5]:
# Init AnnData
# ============

donorsid = m_count.index      # Coger nomes sujetos
genesid = m_count.columns     # Coger nombre genes
zcounts = m_count.to_numpy()  # Coger todos los datos de expression del dataFrame y pasarlos a numpy matrix

annout = ad.AnnData(zcounts, dtype=np.float32)   # init AnnData object
annout.obs_names = [idx for idx in donorsid]     # Actualizar nombres de sujetos  (identificador)
annout.var_names = [idx for idx in genesid]      # Actualizar nombres de genes
#pdb.set_trace()

print('Anndata object created')

Anndata object created


In [6]:
print(annout)

AnnData object with n_obs × n_vars = 40 × 57169


In [7]:
#Import metadata
#================

ifile_meta = r"C:\Users\moren\PycharmProjects\TFM\R\ELA_DFT\decoder_DeSeq2_FTD_FINAL.csv"
meta_obs = pd.read_csv(ifile_meta, header=0)

#meta_obs= meta_obs.set_index('specimenID').sort_index().copy()

meta_obs.index = meta_obs['sample.ID']

meta_obs.head()       

,sample.ID,sample.ID.number,lane.ID,unique.ID,qc.data.dir,group.ID,group2.ID,tissue.ID,BK,GENDER,AGE
sample.ID,,,,,,,,,,,
long10BLACK,long10BLACK,10,unknown,long10BLACK,long10BLACK,TDP,TDP,frontal,1834,M,77
long11BLACK,long11BLACK,11,unknown,long11BLACK,long11BLACK,C9orf72,C9orf72,frontal,414,M,66
long12BLACK,long12BLACK,12,unknown,long12BLACK,long12BLACK,C9orf72,C9orf72,frontal,421,F,69
long13BLACK,long13BLACK,13,unknown,long13BLACK,long13BLACK,C9orf72,C9orf72,frontal,811,M,61
long14BLACK,long14BLACK,14,unknown,long14BLACK,long14BLACK,C9orf72,C9orf72,frontal,1041,M,66


In [8]:
#Loop for matching metadata to annout.X
#======================================

#Get all metadata vars
vars_metadata = list(meta_obs.columns)[1:]

#Inicialize metadata vars
for cvar in vars_metadata:
    annout.obs[cvar] = 0

#Get the metadata row for each donorid    
for donorid in annout.obs.index:
    row = meta_obs.loc[meta_obs.index == donorid,:]

    #Replace var by var for each row    
    for cvar in vars_metadata:
        annout.obs.loc[annout.obs_names == donorid, cvar] = row[cvar]
        
print('Metadata created')

Metadata created


In [9]:
annout.obs.shape
annout.obs

,sample.ID.number,lane.ID,unique.ID,qc.data.dir,group.ID,group2.ID,tissue.ID,BK,GENDER,AGE
long10BLACK,10,unknown,long10BLACK,long10BLACK,TDP,TDP,frontal,1834,M,77
long11BLACK,11,unknown,long11BLACK,long11BLACK,C9orf72,C9orf72,frontal,414,M,66
long12BLACK,12,unknown,long12BLACK,long12BLACK,C9orf72,C9orf72,frontal,421,F,69
long13BLACK,13,unknown,long13BLACK,long13BLACK,C9orf72,C9orf72,frontal,811,M,61
long14BLACK,14,unknown,long14BLACK,long14BLACK,C9orf72,C9orf72,frontal,1041,M,66
long15BLACK,15,unknown,long15BLACK,long15BLACK,C9orf72,C9orf72,frontal,1367,M,69
long16BLACK,16,unknown,long16BLACK,long16BLACK,C9orf72,C9orf72,frontal,1482,F,58
long17BLACK,17,unknown,long17BLACK,long17BLACK,C9orf72,C9orf72,frontal,1636,M,73
long18BLACK,18,unknown,long18BLACK,long18BLACK,C9orf72,C9orf72,frontal,1741,F,57
long19BLACK,19,unknown,long19BLACK,long19BLACK,C9orf72,C9orf72,frontal,1866,M,69


In [10]:
#Adding % of cell types as metadata columns
#===========================================

#Inicialize new cols
cell_types = list(["Ast", "End", "Ex",  "In",  "Mic", "Oli", "Opc", "Per"])

for ctype in cell_types:
    annout.obs[ctype] = 0


#Import %cell type data
#======================
ifile = r"C:\Users\moren\PycharmProjects\TFM\R\ELA_DFT\output_theta\theta.csv"
cell_type_obs = pd.read_csv(ifile, header=0)


cell_type_obs.index = cell_type_obs['ID']
cell_type_obs.head()


#Get all cell types
cell_type_data = list(cell_type_obs.columns)[1:]
    
    
#Add col per ID
#===============

#Get the metadata row for each donorid    
for donorid in annout.obs.index:
    row = cell_type_obs.loc[cell_type_obs.index == donorid,:]

    #Replace var by var for each row    
    for ctype in cell_type_data:
        annout.obs.loc[annout.obs_names == donorid, ctype] = row[ctype]

print('Metadata updated')

Metadata updated


In [11]:
annout.obs.head (10)

,sample.ID.number,lane.ID,unique.ID,qc.data.dir,group.ID,group2.ID,tissue.ID,BK,GENDER,AGE,Ast,End,Ex,In,Mic,Oli,Opc,Per,ID
long10BLACK,10,unknown,long10BLACK,long10BLACK,TDP,TDP,frontal,1834,M,77,0.135844,0.044044,0.649717,0.038132,0.019386,0.111752,0.000002,0.001123,long10BLACK
long11BLACK,11,unknown,long11BLACK,long11BLACK,C9orf72,C9orf72,frontal,414,M,66,0.127826,0.045206,0.609181,0.066464,0.029105,0.121829,0.000001,0.000387,long11BLACK
long12BLACK,12,unknown,long12BLACK,long12BLACK,C9orf72,C9orf72,frontal,421,F,69,0.137009,0.029148,0.704188,0.086086,0.010090,0.031240,0.000001,0.002238,long12BLACK
long13BLACK,13,unknown,long13BLACK,long13BLACK,C9orf72,C9orf72,frontal,811,M,61,0.233917,0.074595,0.485634,0.021129,0.027712,0.144914,0.001586,0.010513,long13BLACK
long14BLACK,14,unknown,long14BLACK,long14BLACK,C9orf72,C9orf72,frontal,1041,M,66,0.151342,0.058873,0.604842,0.051649,0.012300,0.117000,0.000003,0.003992,long14BLACK
long15BLACK,15,unknown,long15BLACK,long15BLACK,C9orf72,C9orf72,frontal,1367,M,69,0.222636,0.070932,0.552721,0.020634,0.028892,0.077743,0.001284,0.025157,long15BLACK
long16BLACK,16,unknown,long16BLACK,long16BLACK,C9orf72,C9orf72,frontal,1482,F,58,0.147972,0.046830,0.633245,0.100055,0.016484,0.051632,0.000001,0.003781,long16BLACK
long17BLACK,17,unknown,long17BLACK,long17BLACK,C9orf72,C9orf72,frontal,1636,M,73,0.273063,0.067992,0.527795,0.002273,0.030971,0.085138,0.004084,0.008683,long17BLACK
long18BLACK,18,unknown,long18BLACK,long18BLACK,C9orf72,C9orf72,frontal,1741,F,57,0.145042,0.069433,0.575040,0.083047,0.018885,0.103219,0.000001,0.005333,long18BLACK
long19BLACK,19,unknown,long19BLACK,long19BLACK,C9orf72,C9orf72,frontal,1866,M,69,0.178508,0.059322,0.523205,0.053026,0.015362,0.166663,0.000251,0.003663,long19BLACK


In [35]:
# Include cell-type-especific gene expression
# ===========================================

ipath_layers = "C:/Users/moren/PycharmProjects/TFM/R/ELA_DFT/output_exp/"
typelist = ['Ast','Ex','In',
            'End','Mic','Oli',
           'Opc','Per']
        
# Each cell type will be included as a layer
# Loop over all celltypes
#========================
for ctype in typelist:
    print(f'Working with type: {ctype}')
    ifile = os.path.join(ipath_layers, ctype+'.csv')
        
    # Read celltype gene exp
    cellexp = pd.read_csv(ifile, index_col=0)
    cellexp.index = cellexp['ID']
    
    # Init new layer
    annout.layers[ctype] = np.zeros_like(annout.X)
    
    # Position intersection of genes between anndat and current celltype
    xy,xpos,ypos = np.intersect1d(annout.var_names, cellexp.columns.values, return_indices=True)
    
    # Iterate over all subjects and put into layer
    for csubj in cellexp.index:
        # Subset current subject
        csubjdf = cellexp.loc[csubj,:]
        # Parse gene expression to correct position
        annout.layers[ctype][annout.obs_names == csubj , xpos] = csubjdf[ypos].values


Working with type: Ast
Working with type: Ex
Working with type: In
Working with type: End
Working with type: Mic
Working with type: Oli
Working with type: Opc
Working with type: Per


In [13]:
#Saving Anndata 
#==============

ofile = join(opath,"ELA_DFT_Anndata.h5ad")
annout.write(ofile, compression="gzip")  # Might take a while b/c we are compressing the file
print(f'Saved AnnData')

Saved AnnData
